# ACHS - PrepData

Author: Alejandro Bañados

Contact: arbanados@uc.cl

First version: Feb-2025

This notebook compiles, clean and make same fast descriptions for work accidents database of ACHS and the temperature data. This proceess data is analyzed later in a R notebook.

In [1]:
#pip install netcdf4
import pandas as pd
import xarray as xr
import geopandas as gpd
import pyarrow
from unidecode import unidecode

## Work related accidents



In [23]:
import pandas as pd

# Path to your Excel file
file_path = 'rawdata/BD-ACHS.xlsx'

# Initialize an empty list to store DataFrames
dfs = []

# Loop through sheet names from 2015 to 2024
for year in range(2015, 2026):  # 2025 is exclusive, so it stops at 2025
    sheet_name = str(year)  # Convert year to string to match sheet names
    try:
        # Read the sheet into a DataFrame
        d = pd.read_excel(file_path, sheet_name=sheet_name)
        
        # Select the required columns
        d = d[['tipo_siniestro_actual_desc', 
               'tipo_siniestro_origen_desc',
               'fecha_presentacion', 
               'fecha_accidente', 
               'hora_accidente',
               'direccion_accidente',
               'descripcion_rubro_achs',
               'puesto_trabajador', 
               'edad', 
               'sexo',
               'diagnostico_1']]
        
        # Optionally, add a column to track the year
        d['Year'] = year
        
        # Append the DataFrame to the list
        dfs.append(d)
    except Exception as e:
        print(f"Sheet '{sheet_name}' not found or could not be read: {e}")

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display a sample of the combined DataFrame (transposed for better readability)
print(combined_df.sample(3).T)

                                                   1988242  \
tipo_siniestro_actual_desc                          No Ley   
tipo_siniestro_origen_desc                         Trabajo   
fecha_presentacion                     2024-05-16 00:00:00   
fecha_accidente                        2024-05-16 00:00:00   
hora_accidente                                    08:50:00   
direccion_accidente                           CORONEL - 08   
descripcion_rubro_achs      FABRICACION DE PROD. METALICOS   
puesto_trabajador                          8332 Conductora   
edad                                                  44.0   
sexo                                             masculino   
diagnostico_1               ROTURA TENDON DE AQUILES AGUDA   
Year                                                  2024   

                                                        672262   \
tipo_siniestro_actual_desc                              Trabajo   
tipo_siniestro_origen_desc                              Tra

C:\Users\aleja\AppData\Local\Temp\ipykernel_32812\1488547469.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


In [48]:
combined_df["Comuna"] = combined_df["direccion_accidente"].str.split("-").str[0].str.strip().str.lower()
combined_df.to_parquet("data/ACHS_accidents.parquet")

In [27]:
combined_df.sample(100000).to_csv("data/mini_sample.csv") #to explore in Tableau

## Temperature
### maximum
We use CR2met that can be download from https://www.cr2.cl/datos-productos-grillados/ this product is a grid that uses ERA5 and ground stations data. We use the commune populated centroids to extract a daily temp for each commune.

In [ ]:
ds=xr.open_dataset(r"C:\Users\aleja\Downloads\CR2MET_v2.0_tmax_day_1979_2020\CR2MET_tmax_v2.0_day_1979_2020_005deg.nc")
mask_time= (ds.time >= pd.to_datetime("2015-01-01")) #analizamos solo desde 2015
ds = ds.where(mask_time, drop=True)
ds

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 1947, lat: 800, lon: 220)
Coordinates:
  * lat      (lat) float32 3kB -56.97 -56.92 -56.88 ... -17.12 -17.08 -17.02
  * lon      (lon) float32 880B -76.97 -76.93 -76.88 ... -66.12 -66.07 -66.03
  * time     (time) datetime64[ns] 16kB 2015-01-01 2015-01-02 ... 2020-04-30
Data variables:
    tmax     (time, lat, lon) float64 3GB nan nan nan nan ... nan nan nan nan
Attributes: (12/20)
    creation_date:       21-Sep-2020 19:15:10
    ls_vars_s2:          tmin tmina tmax tmaxa t2m t2ma t850 t850a t700 td2m ...
    nb_months:           -1  0  1
    av_ens_qtl:          0.0
    agg_metr:            median
    authors:             Boisier et al.
    ...                  ...
    par_nd_stn_min:      10
    par_nd_stn_spl:      50
    par_dlat_min:        2
    par_dlat_max:        15
    history:             Wed Jan 22 12:11:25 2020: ncks -v tmin /home/nvaldeb...
    NCO:                 netCDF Operators version 4.7.9 (Homepage = httpwrite...

In [2]:
centroids = gpd.read_file("data/comunas_pobladas_centroides.gpkg", layer="Centroides")
centroids = centroids.to_crs("EPSG:4326")
centroids

,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,Region,Comuna,Provincia,geometry
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,Marchigüe,Cardenal Caro,POINT (-71.6523 -34.39238)
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,Codegua,Cachapoal,POINT (-70.66278 -34.04137)
2,30,63026.084422,15,8,6103,6,1.448565e+08,76355.326122,Región del Libertador Bernardo O'Higgins,Coinco,Cachapoal,POINT (-70.95749 -34.2699)
3,31,89840.903562,15,8,6104,6,3.256572e+08,108874.623150,Región del Libertador Bernardo O'Higgins,Coltauco,Cachapoal,POINT (-71.07537 -34.28832)
4,78,122626.493264,23,11,9121,9,6.990727e+08,156680.410681,Región de La Araucanía,Cholchol,Cautín,POINT (-72.84563 -38.6016)
...,...,...,...,...,...,...,...,...,...,...,...,...
336,255,130240.555805,19,10,16305,16,8.751006e+08,160459.137929,Región de Ñuble,San Nicolás,Punilla,POINT (-72.15671 -36.51874)
337,253,311061.932415,19,10,16304,16,2.393007e+09,388109.613857,Región de Ñuble,San Fabián,Punilla,POINT (-71.55204 -36.55565)
338,245,159257.906431,19,10,16303,16,7.638173e+08,195593.933727,Región de Ñuble,Ñiquén,Punilla,POINT (-71.81532 -36.28228)
339,243,157467.049667,19,10,16104,16,1.044405e+09,196962.664313,Región de Ñuble,El Carmen,Ñuble,POINT (-72.023 -36.89763)


In [33]:
# Extract lat/lon from points
lat_points = centroids.geometry.y
lon_points = centroids.geometry.x

# Use comuna as dimension name instead of generic "points"
temps = ds["tmax"].sel(
    lat=xr.DataArray(lat_points, dims="comuna"), 
    lon=xr.DataArray(lon_points, dims="comuna"), 
    method="nearest"
)

# Convert to DataFrame
temp = temps.to_dataframe().reset_index()

# Merge "Comunas" column correctly
temp["Comuna"] = temp["comuna"].map(dict(enumerate(centroids["Comuna"])))

# Drop unnecessary "comuna" column (optional)
temp = temp.drop(columns=["comuna"])

# Reorder columns (optional)
temp = temp[["Comuna", "time", "tmax"]]

# Normalize "Comunas" names to Unicode (remove accents & special characters)
temp["Comuna"] = temp["Comuna"].apply(lambda x: unidecode(x).lower())
temp

,Comuna,time,tmax
0,marchigue,2015-01-01,29.265000
1,codegua,2015-01-01,30.410000
2,coinco,2015-01-01,30.708333
3,coltauco,2015-01-01,29.933333
4,cholchol,2015-01-01,22.763333
...,...,...,...
663922,san nicolas,2020-04-30,21.248333
663923,san fabian,2020-04-30,19.916667
663924,niquen,2020-04-30,21.298333
663925,el carmen,2020-04-30,20.768333


In [34]:
temp.to_parquet("data/tmax.parquet")

### mean

In [3]:
ds=xr.open_dataset(r"C:\Users\aleja\Downloads\CR2MET_v2.0_tday_1979_2020\CR2MET_t2m_v2.0_day_1979_2020_005deg.nc")
mask_time= (ds.time >= pd.to_datetime("2015-01-01")) #analizamos solo desde 2015
ds = ds.where(mask_time, drop=True)
ds

<xarray.Dataset> Size: 3GB
Dimensions:  (time: 1947, lat: 800, lon: 220)
Coordinates:
  * lat      (lat) float32 3kB -56.97 -56.92 -56.88 ... -17.12 -17.08 -17.02
  * lon      (lon) float32 880B -76.97 -76.93 -76.88 ... -66.12 -66.07 -66.03
  * time     (time) datetime64[ns] 16kB 2015-01-01 2015-01-02 ... 2020-04-30
Data variables:
    t2m      (time, lat, lon) float64 3GB nan nan nan nan ... nan nan nan nan
Attributes: (12/20)
    creation_date:       21-Sep-2020 19:15:10
    ls_vars_s2:          tmin tmina tmax tmaxa t2m t2ma t850 t850a t700 td2m ...
    nb_months:           -1  0  1
    av_ens_qtl:          0.0
    agg_metr:            median
    authors:             Boisier et al.
    ...                  ...
    par_nd_stn_min:      10
    par_nd_stn_spl:      50
    par_dlat_min:        2
    par_dlat_max:        15
    history:             Wed Jan 22 12:11:25 2020: ncks -v tmin /home/nvaldeb...
    NCO:                 netCDF Operators version 4.7.9 (Homepage = httpwrite...

In [4]:
centroids = gpd.read_file("data/comunas_pobladas_centroides.gpkg", layer="Centroides")
centroids = centroids.to_crs("EPSG:4326")
centroids

# Extract lat/lon from points
lat_points = centroids.geometry.y
lon_points = centroids.geometry.x

# Use comuna as dimension name instead of generic "points"
temps = ds["t2m"].sel(
    lat=xr.DataArray(lat_points, dims="comuna"), 
    lon=xr.DataArray(lon_points, dims="comuna"), 
    method="nearest"
)

# Convert to DataFrame
temp = temps.to_dataframe().reset_index()

# Merge "Comunas" column correctly
temp["Comuna"] = temp["comuna"].map(dict(enumerate(centroids["Comuna"])))

# Drop unnecessary "comuna" column (optional)
temp = temp.drop(columns=["comuna"])

# Reorder columns (optional)
temp = temp[["Comuna", "time", "t2m"]]

# Normalize "Comunas" names to Unicode (remove accents & special characters)
temp["Comuna"] = temp["Comuna"].apply(lambda x: unidecode(x).lower())
temp

temp.to_parquet("data/t2m.parquet")